In [22]:
import mwclient
import time

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Bitcoin']

In [23]:
revs = list(page.revisions())

In [24]:
revs[0]

OrderedDict([('revid', 1210641921),
             ('parentid', 1210639486),
             ('minor', ''),
             ('user', 'AnomieBOT'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=2, tm_mday=27, tm_hour=16, tm_min=46, tm_sec=28, tm_wday=1, tm_yday=58, tm_isdst=-1)),
             ('comment', 'Dating maintenance tags: {{Cn}}')])

In [25]:
revs = sorted(revs, key=lambda rev: rev['timestamp'])

In [26]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [27]:
from transformers import pipeline
sentiment_pipeline = pipeline('sentiment-analysis')

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent['score']
    if sent['label'] == 'NEGATIVE':
        score *= -1
    return score

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [28]:
edits = {}

for rev in revs:
    date = time.strftime('%Y-%m-%d', rev["timestamp"])
    
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
        
    edits[date]['edit_count'] += 1
    
    if 'comment' in rev:
        comment = rev['comment']
        edits[date]['sentiments'].append(find_sentiment(comment))
    
    else:
        pass 

In [30]:
from statistics import mean

for key in edits:
    if len(edits[key]['sentiments']) > 0:
        edits[key]['sentiment'] = mean(edits[key]['sentiments'])
        edits[key]['neg_sentiment'] = len([s for s in edits[key]['sentiments'] if s < 0]) / len(edits[key]['sentiments'])
        
    else:
        edits[key]['sentiment'] = 0
        edits[key]['neg_sentiment'] = 0
        
    del edits[key]['sentiments']
        

In [31]:
edits

{'2009-03-08': {'edit_count': 4,
  'sentiment': -0.5505250245332718,
  'neg_sentiment': 0.75},
 '2009-08-05': {'edit_count': 1,
  'sentiment': 0.748120903968811,
  'neg_sentiment': 0.0},
 '2009-08-06': {'edit_count': 2,
  'sentiment': 0.995745837688446,
  'neg_sentiment': 0.0},
 '2009-08-14': {'edit_count': 1,
  'sentiment': 0.9300212860107422,
  'neg_sentiment': 0.0},
 '2009-10-13': {'edit_count': 2,
  'sentiment': -0.22749948501586914,
  'neg_sentiment': 0.5},
 '2009-11-18': {'edit_count': 1,
  'sentiment': 0.8839507102966309,
  'neg_sentiment': 0.0},
 '2009-12-08': {'edit_count': 1,
  'sentiment': -0.9869275689125061,
  'neg_sentiment': 1.0},
 '2009-12-17': {'edit_count': 1,
  'sentiment': -0.9975171089172363,
  'neg_sentiment': 1.0},
 '2010-02-23': {'edit_count': 1,
  'sentiment': -0.9994946718215942,
  'neg_sentiment': 1.0},
 '2010-03-18': {'edit_count': 1,
  'sentiment': 0.8758783340454102,
  'neg_sentiment': 0.0},
 '2010-04-13': {'edit_count': 4,
  'sentiment': 0.844355911016464

In [32]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient='index')

In [33]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.750000
2009-08-05,1,0.748121,0.000000
2009-08-06,2,0.995746,0.000000
2009-08-14,1,0.930021,0.000000
2009-10-13,2,-0.227499,0.500000
...,...,...,...
2024-02-22,10,-0.056210,0.500000
2024-02-23,7,-0.456403,0.714286
2024-02-25,2,-0.005358,0.500000
2024-02-26,1,-0.996016,1.000000
